# Benchmark Geometry

Set up the PYTHONPATH so it can find the python modules we'll be using.

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))
params_filename = os.path.join(os.pardir, "data", "default_params.json")

Load the default parameters

In [ ]:
import json
with open(params_filename, "r") as fp:
    default_params = json.load(fp)

Import the geometry module

In [ ]:
import geometry as geo

Set the base resolution that will be used throughout

In [ ]:
minres = 10.0

In [ ]:
geo.SlabSpline?

In [ ]:
coast_distance = 0
extra_width = 0
lc_depth = 40
uc_depth = 15
io_depth = 139
xs = [0.0, 140.0, 240.0, 400.0]
ys = [0.0, -70.0, -120.0, -200.0]
res = [1*minres if y >= -default_params["slab_diag_depth2"] else 3*minres for y in ys]
res

In [ ]:
# set up the surface ids for the slab depending on depth
sids = []
for y in ys[1:]:
    if y >= -lc_depth: 
        sid = default_params['fault_sid']
    elif y >= -default_params['slab_diag_depth1']:
        sid = default_params['slab_sid']
    elif y >= -default_params['slab_diag_depth2']:
        sid = default_params['slab_diag_sid']
    else:
        sid = default_params['slab_sid']
    sids.append(sid)
sids

In [ ]:
# set up the slab spline
slab = geo.SlabSpline(xs, ys, res=res, sid=sids, name="Slab")
print(len(slab.points))

In [ ]:
# adding the coupling depths may or may not be necessary
# depending on if they were included in the slab spline data already or not
# the slab class should ignore them if they aren't necessary
slab.addpoint(default_params['partial_coupling_depth'], "Slab::PartialCouplingDepth", 
              res=default_params['partial_coupling_depth_res_fact']*minres, 
              sid=default_params['slab_diag_sid'])
slab.addpoint(default_params['full_coupling_depth'], "Slab::FullCouplingDepth", 
              res=default_params['full_coupling_depth_res_fact']*minres, 
              sid=default_params['slab_diag_sid'])
# add the slab detector point
slab.addpoint(default_params["slab_det_depth"], "Slab::DetectorPoint", 
              res=default_params["full_coupling_depth_res_fact"]*minres,
              sid=default_params['slab_diag_sid'])
print(len(slab.points))

In [ ]:
geo.SubductionGeometry?

In [ ]:
geom = geo.SubductionGeometry(slab, **default_params, 
                              wedge_side_top_res=default_params["wedge_side_top_res_fact"]*minres,
                              wedge_side_base_res=default_params["wedge_side_base_res_fact"]*minres,
                              slab_side_base_res=default_params["slab_side_base_res_fact"]*minres,
                              coast_res=default_params["coast_res_fact"]*minres,
                              coast_distance=coast_distance, extra_width=extra_width)
geom.plot()

In [ ]:
# add a lower crust
geom.addcrustlayer(lc_depth, "LowerCrust", 
                   sid=default_params["lc_base_sid"], rid=default_params["lc_rid"],
                   slab_res=default_params["lc_slab_res_fact"]*minres,
                   side_res=default_params["lc_side_res_fact"]*minres,
                   slab_sid=default_params["fault_sid"],
                   side_sid=default_params["lc_side_sid"])
geom.plot()

In [ ]:
geom.addcrustlayer(uc_depth, "UpperCrust", 
                   sid=default_params["uc_base_sid"], rid=default_params["uc_rid"],
                   slab_res=default_params["uc_slab_res_fact"]*minres,
                   side_res=default_params["uc_side_res_fact"]*minres,
                   slab_sid=default_params["fault_sid"],
                   side_sid=default_params["uc_side_sid"])
geom.plot()

In [ ]:
# add the pre-defined in-out point
geom.addwedgesidepoint(io_depth, "WedgeSide::InOut", line_name="UpperWedgeSide", 
                       res=default_params["io_depth_res_fact"]*minres, 
                       sid=default_params["upper_wedge_side_sid"])
geom.plot()

In [ ]:
# add wedge dividers for the diagnostics
geom.addwedgedivider(default_params["slab_diag_depth1"], "ColdCorner", 
                     slab_res=default_params["slab_diag_res2_fact"]*minres, 
                     top_res=default_params["slab_diag_res2_fact"]*minres,
                     rid=default_params["wedge_rid"], 
                     slab_sid=default_params["slab_sid"])
geom.plot()

In [ ]:
# add wedge dividers for the diagnostics
geom.addwedgedivider(default_params["slab_diag_depth2"], "WedgeFocused", 
                     slab_res=default_params["slab_diag_res1_fact"]*minres, 
                     top_res=default_params["slab_diag_res1_fact"]*minres,
                     rid=default_params["wedge_diag_rid"], 
                     slab_sid=default_params["slab_diag_sid"])
geom.plot()

In [ ]:
geom.plot(label_sids=True, label_rids=True)

In [ ]:
geom.writegeofile('benchmark.geo_unrolled')
mesh, ct, ft = geom.generatemesh()

In [ ]:
import dolfinx
import numpy as np
import pyvista
pyvista.start_xvfb()
from dolfinx.plot import vtk_mesh
topology, cell_types, x = vtk_mesh(mesh)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
num_local_cells = mesh.topology.index_map(mesh.topology.dim).size_local
grid.cell_data["Marker"] = ct.values[ct.indices<num_local_cells]
grid.set_active_scalars("Marker")

p = pyvista.Plotter(window_size=[800, 800])
p.add_mesh(grid, show_edges=True)
p.view_xy()
p.show()

In [ ]:
tdim = mesh.topology.dim
fdim = tdim - 1
fcc = mesh.topology.connectivity(fdim, tdim)
cft = np.zeros_like(ct.values)
for f in range(ft.values.size):
    for c in fcc.links(ft.indices[f]):
        cft[c] = ft.values[f]
import pyvista
pyvista.start_xvfb()
from dolfinx.plot import vtk_mesh
topology, cell_types, x = vtk_mesh(mesh)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Marker"] = cft
grid.set_active_scalars("Marker")

p = pyvista.Plotter(window_size=[800, 800])
p.add_mesh(grid, show_edges=True)
p.view_xy()
p.show()